In [1]:
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score , classification_report
import numpy as np


In [2]:
df = pd.read_csv('insurance.csv')

In [3]:
df.sample(5)

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category
12,42,95.2,1.78,17.58,True,Chandigarh,freelancer,High
97,52,60.8,1.80,44.86,False,Hyderabad,freelancer,Low
24,50,54.2,1.66,18.60,False,Mysore,private_job,Medium
25,59,60.2,1.55,30.00,False,Mysore,government_job,Low
21,69,92.7,1.84,2.91,False,Jalandhar,retired,High


In [4]:
df_feat = df.copy()

In [5]:
# Feature 1 :BMI
df_feat['bmi'] = df_feat['weight'] / (df_feat['height'] ** 2)

In [6]:
# Feature 2 : Age Group
def age_group(age):
    if age < 25:
        return 'young'
    elif 25 <= age < 45:
        return 'adult'
    elif 45 <= age < 60:
        return 'middle_aged'        
    else:   
        return 'senior'

In [7]:
df_feat['age_group'] = df_feat['age'].apply(age_group)

In [8]:
# Feature 3 : Lifestyle Risk
def lifestyle_risk(row):
    if row['smoker'] and row['bmi'] >30 :
        return 'high'
    elif row['smoker'] or row['bmi'] > 27:
        return 'medium'
    else:
        return 'low'

In [9]:
df_feat['lifestyle_risk'] = df_feat.apply(lifestyle_risk, axis=1)

In [10]:
tier_1_cities = ["Mumbai", "Delhi", "Bangalore", "Chennai", "Kolkata", "Hyderabad", "Pune"]
tier_2_cities = [
    "Jaipur", "Chandigarh", "Indore", "Lucknow", "Patna", "Ranchi", "Visakhapatnam", "Coimbatore",
    "Bhopal", "Nagpur", "Vadodara", "Surat", "Rajkot", "Jodhpur", "Raipur", "Amritsar", "Varanasi",
    "Agra", "Dehradun", "Mysore", "Jabalpur", "Guwahati", "Thiruvananthapuram", "Ludhiana", "Nashik",
    "Allahabad", "Udaipur", "Aurangabad", "Hubli", "Belgaum", "Salem", "Vijayawada", "Tiruchirappalli",
    "Bhavnagar", "Gwalior", "Dhanbad", "Bareilly", "Aligarh", "Gaya", "Kozhikode", "Warangal",
    "Kolhapur", "Bilaspur", "Jalandhar", "Noida", "Guntur", "Asansol", "Siliguri"
]

In [11]:
# Feature 4: City Tier
def city_tier(city):
    if city in tier_1_cities:
        return 1
    elif city in tier_2_cities:
        return 2
    else:
        return 3

In [12]:
df_feat['city_tier'] = df_feat['city'].apply(city_tier)

In [13]:


df_feat.drop(columns=['age', 'weight', 'height', 'smoker', 'city'])[['income_lpa', 'occupation', 'bmi', 'age_group', 'lifestyle_risk', 'city_tier', 'insurance_premium_category']].sample(5)
     


,income_lpa,occupation,bmi,age_group,lifestyle_risk,city_tier,insurance_premium_category
39,11.99,unemployed,35.643424,middle_aged,high,1,High
60,49.94,unemployed,30.920912,adult,high,2,High
78,14.74,freelancer,27.932798,middle_aged,medium,2,High
27,34.33,private_job,35.159702,middle_aged,medium,2,Medium
51,28.95,private_job,38.827923,middle_aged,high,2,High


In [14]:
# Select features and target 
x = df_feat[['bmi' , 'age_group', 'lifestyle_risk', 'city_tier', 'income_lpa', 'occupation']]
y = df_feat['insurance_premium_category']

In [15]:
x

,bmi,age_group,lifestyle_risk,city_tier,income_lpa,occupation
0,49.227482,senior,medium,2,2.92000,retired
1,30.189017,adult,medium,1,34.28000,freelancer
2,21.118382,adult,low,2,36.64000,freelancer
3,45.535900,young,high,1,3.34000,student
4,24.296875,senior,medium,2,3.94000,retired
...,...,...,...,...,...,...
95,21.420747,adult,low,2,19.64000,business_owner
96,47.984483,adult,medium,1,34.01000,private_job
97,18.765432,middle_aged,low,1,44.86000,freelancer
98,30.521676,adult,medium,1,28.30000,business_owner


In [16]:
y

0       High
1        Low
2        Low
3     Medium
4       High
       ...  
95       Low
96       Low
97       Low
98       Low
99       Low
Name: insurance_premium_category, Length: 100, dtype: object

In [17]:
# Define categorical and numerical features
categorical_features = ['age_group', 'lifestyle_risk', 'occupation' , 'city_tier']
numerical_features = ['bmi', 'income_lpa']

In [18]:
# Create column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features),
        ('num', 'passthrough', numerical_features)
        
    ]
)

In [19]:
# Create a pipeline with preprocessing and random forest classifier
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier( random_state=42))
])

In [20]:
# Split data and train the model
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
pipeline.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  ['age_group',
                                                   'lifestyle_risk',
                                                   'occupation', 'city_tier']),
                                                 ('num', 'passthrough',
                                                  ['bmi', 'income_lpa'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [21]:
# Predict and evaluate
y_pred = pipeline.predict(x_test)
accuracy_score(y_test, y_pred)

0.9

In [22]:
x_test.sample(5)

,bmi,age_group,lifestyle_risk,city_tier,income_lpa,occupation
39,35.643424,middle_aged,high,1,11.99,unemployed
10,22.949982,adult,medium,1,32.78,business_owner
78,27.932798,middle_aged,medium,2,14.74,freelancer
82,17.874812,adult,low,1,12.96,unemployed
51,38.827923,middle_aged,high,2,28.95,private_job


In [23]:
import pickle

# Save the trained pipeline using pickle
pickle_model_path = 'model.pkl'
with open(pickle_model_path, 'wb') as f:
    pickle.dump(pipeline, f)